In [1]:
import sqlite3
from typing import Optional

import pandas as pd

In [29]:
import pandas as pd

# Create a dummy DataFrame
data = {
    'date': ['2023-01-01', '2023-01-02', '2023-01-03', '2023-01-04'],
    'revenue': [1000, 1100, 1200, 1300],
    'cost_of_revenue': [700, 750, 800, 850]
}

df = pd.DataFrame(data)

# Convert the 'date' column to a datetime type if it's not already
df['date'] = pd.to_datetime(df['date'])

# Calculate value change for selected columns
columns_to_convert = ['revenue', 'cost_of_revenue']

for column in columns_to_convert:
    df[f'{column}_value_change'] = df[column] - df[column].shift(1)

# Replace NaN values with zero in the value change columns
df.fillna(0, inplace=True)

# Display the updated DataFrame
print(df)


        date  revenue  cost_of_revenue  revenue_value_change  \
0 2023-01-01     1000              700                   0.0   
1 2023-01-02     1100              750                 100.0   
2 2023-01-03     1200              800                 100.0   
3 2023-01-04     1300              850                 100.0   

   cost_of_revenue_value_change  
0                           0.0  
1                          50.0  
2                          50.0  
3                          50.0  


## Create the dataframe which contains all the historical fundamental data of a stock

In [5]:
# Create stocks fundamental df
conn = sqlite3.connect('/Users/orestis/MyProjects/InvestorAPI/app/database/ibd.db')

In [6]:
def get_stock_fundamental_df(symbol: str) -> pd.DataFrame:
    query = f'''
        SELECT income_statement.*, balance_sheet.*, cash_flow.*, stock_overview.sector
        FROM income_statement
        INNER JOIN balance_sheet
        ON income_statement.fiscal_date_ending = balance_sheet.fiscal_date_ending  AND balance_sheet.symbol = '{symbol}'
        INNER JOIN cash_flow
        ON income_statement.fiscal_date_ending = cash_flow.fiscal_date_ending  AND cash_flow.symbol = '{symbol}'
        INNER JOIN stock_overview
        ON income_statement.symbol = stock_overview.symbol  AND stock_overview.symbol = '{symbol}'
        WHERE income_statement.symbol = '{symbol}'
        ORDER BY DATE(income_statement.fiscal_date_ending)
    '''
    stock_df = pd.read_sql(query, conn)
    
    # Drop columns with duplicated names
    stock_df = stock_df.loc[:, ~stock_df.columns.duplicated()]

    # List of columns to convert to float
    columns_to_convert = stock_df.columns.difference(
        ['symbol', 'fiscal_date_ending', 'reported_currency', 'sector']
    )

    # Convert selected columns to float
    stock_df[columns_to_convert] = stock_df[columns_to_convert].astype(float)
    # Fill NaN values with zero for the selected columns
    stock_df[columns_to_convert] = stock_df[columns_to_convert].fillna(0)

    for column in columns_to_convert:
        if column not in ['change_in_cash_and_cash_equivalents', 'change_in_exchange_rate']:
            new_column_name = f'{column}_value_change'
            stock_df[new_column_name] = stock_df[column] - stock_df[column].shift(1)

    return stock_df


In [7]:
stock_df = get_stock_fundamental_df('NVDA')
#stock_df['total_revenue_change'] = stock_df['total_revenue'].pct_change() * 100

/var/folders/fx/63_2h8w151976nqmhc4tbk7c0000gn/T/ipykernel_13218/1590108327.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_df[new_column_name] = stock_df[column] - stock_df[column].shift(1)
/var/folders/fx/63_2h8w151976nqmhc4tbk7c0000gn/T/ipykernel_13218/1590108327.py:32: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_df[new_column_name] = stock_df[column] - stock_df[column].shift(1)
/var/folders/fx/63_2h8w151976nqmhc4tbk7c0000gn/T/ipykernel_13218/1590108327.py:32: PerformanceWarning: DataFrame is highly fragmen

In [8]:
def get_stock_time_series_df(symbol: str) -> pd.DataFrame:
    query = f'''
    SELECT  *
    FROM stock_time_series
    WHERE symbol = '{symbol}'
    ORDER BY registered_date_ts DESC
    '''
    
    stock_time_series_df = pd.read_sql(query, conn)
    return stock_time_series_df


In [9]:
stock_time_series_df = get_stock_time_series_df("NVDA")

In [15]:
def calculate_time_series_avg_value(
    start_date: str,
    time_series_df: pd.DataFrame,
    target_column: str,
    days: int = 95
) -> Optional[int]:
    """
    Given a start calculate what was the avg value
    between <start_date> and <start_date> + <days> time
    """
    if days < 0:
        lower_bound = pd.Timestamp(start_date) - pd.DateOffset(days=abs(days))
        upper_bound = pd.Timestamp(start_date) 
    else:
        lower_bound = pd.Timestamp(start_date)
        upper_bound = lower_bound + pd.DateOffset(days=days)
    
    # Filter the DataFrame
    filtered_df = time_series_df[(time_series_df['registered_date_ts'] >= lower_bound.timestamp()) & (time_series_df['registered_date_ts'] <= upper_bound.timestamp())]
    print("START DATE", start_date)
    print(filtered_df)
    if len(filtered_df) == 0:
        return None

    average_value = filtered_df[target_column].mean()
    return average_value


In [12]:
stock_df

,symbol,fiscal_date_ending,reported_currency,gross_profit,total_revenue,cost_of_revenue,cost_of_goods_and_services_sold,operating_income,selling_general_and_administrative,research_and_development,...,short_term_investments_value_change,total_assets_value_change,total_current_assets_value_change,total_current_liabilities_value_change,total_liabilities_value_change,total_non_current_assets_value_change,total_non_current_liabilities_value_change,total_revenue_value_change,total_shareholder_equity_value_change,treasury_stock_value_change
0,NVDA,2019-01-27,USD,1.207000e+09,2.205000e+09,9.980000e+08,9.980000e+08,2.940000e+08,266000000.0,6.470000e+08,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NVDA,2019-04-28,USD,1.296000e+09,2.220000e+09,9.240000e+08,9.240000e+08,3.580000e+08,264000000.0,6.740000e+08,...,-1.610000e+09,7.290000e+08,7.200000e+07,-1.460000e+08,3.670000e+08,1.100000e+07,-2.800000e+07,1.500000e+07,3.620000e+08,2.110000e+08
2,NVDA,2019-07-28,USD,1.541000e+09,2.579000e+09,1.038000e+09,1.038000e+09,5.710000e+08,266000000.0,7.040000e+08,...,-3.660000e+09,7.540000e+08,7.620000e+08,1.340000e+08,1.220000e+08,-6.000000e+07,5.280000e+08,3.590000e+08,6.320000e+08,5.000000e+07
3,NVDA,2019-10-27,USD,1.916000e+09,3.014000e+09,1.098000e+09,1.098000e+09,9.270000e+08,277000000.0,7.120000e+08,...,-1.366000e+09,1.035000e+09,1.029000e+09,1.580000e+08,1.570000e+08,1.900000e+07,1.300000e+07,4.350000e+08,8.780000e+08,2.020000e+08
4,NVDA,2020-01-26,USD,2.015000e+09,3.105000e+09,1.090000e+09,1.090000e+09,9.900000e+08,287000000.0,7.380000e+08,...,-3.000000e+06,1.505000e+09,1.270000e+09,3.090000e+08,5.150000e+08,3.720000e+08,2.290000e+08,9.100000e+07,9.900000e+08,8.800000e+07
5,NVDA,2020-04-26,USD,2.004000e+09,3.080000e+09,1.076000e+09,1.076000e+09,9.760000e+08,293000000.0,7.350000e+08,...,8.590000e+08,5.939000e+09,5.894000e+09,1.190000e+08,5.044000e+09,4.740000e+08,-2.500000e+07,-2.500000e+07,8.950000e+08,2.220000e+08
6,NVDA,2020-07-26,USD,2.275000e+09,3.866000e+09,1.591000e+09,1.591000e+09,6.510000e+08,627000000.0,9.970000e+08,...,6.847000e+09,1.926000e+09,-4.903000e+09,5.070000e+08,1.111000e+09,6.791000e+09,1.099000e+09,7.860000e+08,8.150000e+08,1.960000e+08
7,NVDA,2020-10-25,USD,2.960000e+09,4.726000e+09,1.766000e+09,1.766000e+09,1.398000e+09,515000000.0,1.047000e+09,...,1.810000e+08,1.701000e+09,-2.880000e+08,1.259000e+09,2.810000e+08,2.252000e+09,6.021000e+09,8.600000e+08,1.420000e+09,2.980000e+08
8,NVDA,2021-01-31,USD,3.156000e+09,5.003000e+09,1.847000e+09,1.846000e+09,1.507000e+09,503000000.0,1.146000e+09,...,2.826000e+09,1.910000e+09,1.662000e+09,2.560000e+08,3.510000e+08,2.860000e+08,1.760000e+08,2.770000e+08,1.559000e+09,2.260000e+08
9,NVDA,2021-05-02,USD,3.629000e+09,5.661000e+09,2.032000e+09,2.032000e+09,1.956000e+09,520000000.0,1.153000e+09,...,9.750000e+08,2.005000e+09,2.072000e+09,7.900000e+07,1.240000e+08,-2.110000e+08,-1.228000e+09,6.580000e+08,1.881000e+09,4.860000e+08


In [16]:
stock_df['avg_three_months_price'] = stock_df['fiscal_date_ending'].apply(
    calculate_time_series_avg_value,
    target_column='close_price',
    time_series_df=stock_time_series_df,
    days=-95
)


START DATE 2019-01-27
   symbol  open_price  high_price  low_price  close_price       volume  \
58   NVDA      172.60      174.68     124.46       133.50  311279473.0   
59   NVDA      212.30      222.00     133.31       163.43  404058585.0   
60   NVDA      284.16      292.76     176.01       210.83  324449489.0   

    dividend_amount registered_date  registered_date_ts  
58             0.00      31-12-2018          1546207200  
59             0.16      30-11-2018          1543528800  
60             0.00      31-10-2018          1540936800  
START DATE 2019-04-28
   symbol  open_price  high_price  low_price  close_price       volume  \
55   NVDA      156.27      185.00     144.80       179.56  301712810.0   
56   NVDA      144.50      165.28     142.58       154.26  286143490.0   
57   NVDA      130.64      160.88     127.69       143.75  421791107.0   

    dividend_amount registered_date  registered_date_ts  
55             0.00      29-03-2019          1553810400  
56            

In [17]:
stock_df['avg_next_three_months_price'] = stock_df['fiscal_date_ending'].apply(
    calculate_time_series_avg_value,
    target_column='close_price',
    time_series_df=stock_time_series_df,
)


START DATE 2019-01-27
   symbol  open_price  high_price  low_price  close_price       volume  \
54   NVDA      183.26      193.47     173.30       181.00  236076606.0   
55   NVDA      156.27      185.00     144.80       179.56  301712810.0   
56   NVDA      144.50      165.28     142.58       154.26  286143490.0   
57   NVDA      130.64      160.88     127.69       143.75  421791107.0   

    dividend_amount registered_date  registered_date_ts  
54             0.00      30-04-2019          1556571600  
55             0.00      29-03-2019          1553810400  
56             0.16      28-02-2019          1551304800  
57             0.00      31-01-2019          1548885600  
START DATE 2019-04-28
   symbol  open_price  high_price  low_price  close_price       volume  \
51   NVDA      172.57      178.89     155.00       168.72  210468645.0   
52   NVDA      135.91      165.37     132.60       164.23  205833345.0   
53   NVDA      183.11      184.87     135.39       135.46  279518852.0   

/var/folders/fx/63_2h8w151976nqmhc4tbk7c0000gn/T/ipykernel_13218/3983939058.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  stock_df['avg_next_three_months_price'] = stock_df['fiscal_date_ending'].apply(


In [28]:
stock_df[['short_term_debt', 'short_term_debt_pct_change']]

,short_term_debt,short_term_debt_pct_change
0,0.000000e+00,NaN
1,0.000000e+00,NaN
2,0.000000e+00,NaN
3,0.000000e+00,NaN
4,0.000000e+00,NaN
5,0.000000e+00,NaN
6,0.000000e+00,NaN
7,0.000000e+00,NaN
8,0.000000e+00,NaN
9,1.134000e+09,inf


## Create dataframe that contains interest rates historical data

In [5]:
# Create the interest rates df
query = '''
SELECT  *
FROM economic_indicator_time_series
WHERE indicator_name = 'Interest_Rate'
'''

interest_rate_df = pd.read_sql(query, conn)
interest_rate_df

,indicator_name,value,unit,registered_date,registered_date_ts
0,Interest_Rate,5.12,percent,01-07-2023,1688158800
1,Interest_Rate,5.08,percent,01-06-2023,1685566800
2,Interest_Rate,5.06,percent,01-05-2023,1682888400
3,Interest_Rate,4.83,percent,01-04-2023,1680296400
4,Interest_Rate,4.65,percent,01-03-2023,1677621600
...,...,...,...,...,...
824,Interest_Rate,0.83,percent,01-11-1954,-478663200
825,Interest_Rate,0.85,percent,01-10-1954,-481341600
826,Interest_Rate,1.07,percent,01-09-1954,-483933600
827,Interest_Rate,1.22,percent,01-08-1954,-486612000


## Create dataframe that contains treasury yield(10 year) historical data

In [6]:
query = '''
SELECT  *
FROM economic_indicator_time_series
WHERE indicator_name = 'Treasury_Yield'
'''

treasury_yield_df = pd.read_sql(query, conn)
treasury_yield_df

,indicator_name,value,unit,registered_date,registered_date_ts
0,Treasury_Yield,3.90,percent,01-07-2023,1688158800
1,Treasury_Yield,3.75,percent,01-06-2023,1685566800
2,Treasury_Yield,3.57,percent,01-05-2023,1682888400
3,Treasury_Yield,3.46,percent,01-04-2023,1680296400
4,Treasury_Yield,3.66,percent,01-03-2023,1677621600
...,...,...,...,...,...
839,Treasury_Yield,2.95,percent,01-08-1953,-518148000
840,Treasury_Yield,2.93,percent,01-07-1953,-520826400
841,Treasury_Yield,3.11,percent,01-06-1953,-523418400
842,Treasury_Yield,3.05,percent,01-05-1953,-526096800


## Create dataframe that contains global commodities index historical data

In [7]:
query = '''
SELECT  *
FROM economic_indicator_time_series
WHERE indicator_name = 'Global_Commodities_Index'
'''

commodities_index_df = pd.read_sql(query, conn)
commodities_index_df

,indicator_name,value,unit,registered_date,registered_date_ts
0,Global_Commodities_Index,154.695420,index 2016=100,01-06-2023,1685566800
1,Global_Commodities_Index,157.292251,index 2016=100,01-05-2023,1682888400
2,Global_Commodities_Index,170.972547,index 2016=100,01-04-2023,1680296400
3,Global_Commodities_Index,168.294185,index 2016=100,01-03-2023,1677621600
4,Global_Commodities_Index,174.564801,index 2016=100,01-02-2023,1675202400
...,...,...,...,...,...
241,Global_Commodities_Index,63.065977,index 2016=100,01-05-2003,1051736400
242,Global_Commodities_Index,61.887199,index 2016=100,01-04-2003,1049144400
243,Global_Commodities_Index,65.782768,index 2016=100,01-03-2003,1046469600
244,Global_Commodities_Index,70.229514,index 2016=100,01-02-2003,1044050400


## Create dataframe that contains Unemployment historical data

In [8]:
query = '''
SELECT  *
FROM economic_indicator_time_series
WHERE indicator_name = 'Unemployment'
'''

unemployment_df = pd.read_sql(query, conn)
unemployment_df

,indicator_name,value,unit,registered_date,registered_date_ts
0,Unemployment,3.5,percent,01-07-2023,1688158800
1,Unemployment,3.6,percent,01-06-2023,1685566800
2,Unemployment,3.7,percent,01-05-2023,1682888400
3,Unemployment,3.4,percent,01-04-2023,1680296400
4,Unemployment,3.5,percent,01-03-2023,1677621600
...,...,...,...,...,...
902,Unemployment,3.5,percent,01-05-1948,-683863200
903,Unemployment,3.9,percent,01-04-1948,-686455200
904,Unemployment,4.0,percent,01-03-1948,-689133600
905,Unemployment,3.8,percent,01-02-1948,-691639200


## Create dataframe that contains Inflation historical data(yearly)

In [9]:
query = '''
SELECT  *
FROM economic_indicator_time_series
WHERE indicator_name = 'Inflation'
ORDER BY registered_date_ts DESC
'''

inflation_df = pd.read_sql(query, conn)
inflation_df

,indicator_name,value,unit,registered_date,registered_date_ts
0,Inflation,4.642850,percent,01-01-2023,1693083600
1,Inflation,8.002800,percent,01-01-2022,1640988000
2,Inflation,4.697859,percent,01-01-2021,1609452000
3,Inflation,1.233584,percent,01-01-2020,1577829600
4,Inflation,1.812210,percent,01-01-2019,1546293600
...,...,...,...,...,...
59,Inflation,1.278912,percent,01-01-1964,-189396000
60,Inflation,1.239669,percent,01-01-1963,-220932000
61,Inflation,1.198773,percent,01-01-1962,-252468000
62,Inflation,1.070724,percent,01-01-1961,-284004000


## Create dataframe that contains the stock time series

In [10]:
def get_stock_time_series_df(symbol: str) -> pd.DataFrame:
    query = f'''
    SELECT  *
    FROM stock_time_series
    WHERE symbol = '{symbol}'
    ORDER BY registered_date_ts DESC
    '''
    
    stock_time_series_df = pd.read_sql(query, conn)
    return stock_time_series_df

In [11]:
stock_time_series_df = get_stock_time_series_df('NVDA')
stock_time_series_df

,symbol,open_price,high_price,low_price,close_price,volume,dividend_amount,registered_date,registered_date_ts
0,NVDA,497.620,498.00,481.416,485.09,4.638303e+07,0.00,01-09-2023,1693515600
1,NVDA,464.595,502.66,403.110,493.55,1.363638e+09,0.00,31-08-2023,1693429200
2,NVDA,425.170,480.88,413.460,467.29,8.709012e+08,0.00,31-07-2023,1690750800
3,NVDA,384.890,439.90,373.560,423.02,1.052802e+09,0.04,30-06-2023,1688072400
4,NVDA,278.400,419.38,272.400,378.34,1.169407e+09,0.00,31-05-2023,1685480400
...,...,...,...,...,...,...,...,...,...
281,NVDA,83.720,110.40,57.000,89.13,1.350360e+07,0.00,28-04-2000,956869200
282,NVDA,64.000,150.00,57.500,84.48,3.656700e+07,0.00,31-03-2000,954450000
283,NVDA,37.130,68.25,35.750,64.00,1.908510e+07,0.00,29-02-2000,951775200
284,NVDA,47.250,48.25,35.000,37.06,1.029470e+07,0.00,31-01-2000,949269600


## Helper function to calculate average value from given time series in a given timeframe

In [12]:
def calculate_time_series_avg_value(
    start_date: str,
    time_series_df: pd.DataFrame,
    target_column: str,
    days: int = 31
) -> Optional[int]:
    """
    Given a start calculate what was the avg value
    between <start_date> and <start_date> + <days> time
    """
    lower_bound = pd.Timestamp(start_date)
    
    upper_bound = lower_bound + pd.DateOffset(days=days)
    
    # Filter the DataFrame
    filtered_df = time_series_df[(time_series_df['registered_date_ts'] >= lower_bound.timestamp()) & (time_series_df['registered_date_ts'] <= upper_bound.timestamp())]
    
    if len(filtered_df) == 0:
        return None

    average_value = filtered_df[target_column].mean()
    return average_value

## Helper function to return inflation of given year

In [13]:
def get_inflation_value_by_date(date_string: str, inflation_df: pd.DataFrame) -> Optional[float]:
    try:
        date_obj = pd.to_datetime(date_string, format='%Y-%m-%d')
        target_year = date_obj.year
        inflation_df['register_date_pandas_dt'] = pd.to_datetime(inflation_df['registered_date'], format='%d-%m-%Y')
        selected_row = inflation_df[inflation_df['register_date_pandas_dt'].dt.year == target_year]
        if not selected_row.empty:
            return selected_row['value'].iloc[0]
        else:
            return None
    except (ValueError, KeyError):
        return None

In [14]:
result = get_inflation_value_by_date('2023-12-31', inflation_df)
print("Result is:", result)

Result is: 4.64285


## Create new columns with average values of economic indicators and inflation of given year

In [15]:
temp_stock_df = stock_df.copy()
temp_stock_df['avg_interest_rate'] = temp_stock_df['fiscal_date_ending'].apply(
                                                          calculate_time_series_avg_value,
                                                          target_column='value',
                                                          time_series_df=interest_rate_df,
                                                          days=40
                                                        )

In [16]:
temp_stock_df['avg_treasury_yield'] = temp_stock_df['fiscal_date_ending'].apply(
                                                          calculate_time_series_avg_value,
                                                          target_column='value',
                                                          time_series_df=treasury_yield_df,
                                                          days=40
                                                        )

In [17]:
temp_stock_df['avg_unemployment_rate'] = temp_stock_df['fiscal_date_ending'].apply(
                                                          calculate_time_series_avg_value,
                                                          target_column='value',
                                                          time_series_df=unemployment_df,
                                                          days=40
                                                        )

In [18]:
temp_stock_df['avg_global_commodities_index_value'] = temp_stock_df['fiscal_date_ending'].apply(
                                                          calculate_time_series_avg_value,
                                                          target_column='value',
                                                          time_series_df=commodities_index_df,
                                                          days=40
                                                        )

In [19]:
temp_stock_df['inflation'] = temp_stock_df['fiscal_date_ending'].apply(
                                                          get_inflation_value_by_date,
                                                          inflation_df=inflation_df,
                                                        )

## Create volume column

In [20]:
temp_stock_df['volume'] = temp_stock_df['fiscal_date_ending'].apply(
                                                          calculate_time_series_avg_value,
                                                          target_column='volume',
                                                          time_series_df=stock_time_series_df,
                                                          days=40
                                                        )

## Create target column

In [21]:
temp_stock_df['price'] = temp_stock_df['fiscal_date_ending'].apply(
                                                          calculate_time_series_avg_value,
                                                          target_column='close_price',
                                                          time_series_df=stock_time_series_df,
                                                          days=40
                                                        )

In [22]:
temp_stock_df.head()

,symbol,fiscal_date_ending,reported_currency,gross_profit,total_revenue,cost_of_revenue,cost_of_goods_and_services_sold,operating_income,selling_general_and_administrative,research_and_development,...,proceeds_from_sale_of_treasury_stock,change_in_cash_and_cash_equivalents,change_in_exchange_rate,avg_interest_rate,avg_treasury_yield,avg_unemployment_rate,avg_global_commodities_index_value,inflation,volume,price
0,NVDA,2023-07-30,USD,9.462000e+09,1.350700e+10,4.045000e+09,3.430000e+08,6.800000e+09,622000000.0,2.040000e+09,...,None,8.030000e+08,None,NaN,NaN,NaN,NaN,4.64285,7.603075e+08,481.976667
1,NVDA,2023-04-30,USD,4.648000e+09,7.192000e+09,2.544000e+09,2.544000e+09,2.140000e+09,633000000.0,1.875000e+09,...,None,1.690000e+09,None,5.070,3.660,3.65,155.993835,4.64285,1.169407e+09,378.340000
2,NVDA,2023-01-29,USD,3.833000e+09,6.051000e+09,2.218000e+09,2.218000e+09,1.256000e+09,625000000.0,1.952000e+09,...,None,NaN,None,4.610,3.705,3.55,171.429493,4.64285,9.924524e+08,213.765000
3,NVDA,2022-10-30,USD,3.177000e+09,5.931000e+09,2.754000e+09,2.754000e+09,6.010000e+08,631000000.0,1.945000e+09,...,None,NaN,None,3.940,3.755,3.55,193.215259,8.00280,1.157644e+09,152.100000
4,NVDA,2022-07-31,USD,2.915000e+09,6.704000e+09,3.789000e+09,5.700000e+08,4.990000e+08,592000000.0,1.824000e+09,...,None,NaN,None,2.445,3.210,3.60,230.968689,8.00280,1.206915e+09,150.940000


In [23]:
stock_time_series_df.head(10)

,symbol,open_price,high_price,low_price,close_price,volume,dividend_amount,registered_date,registered_date_ts
0,NVDA,497.620,498.0000,481.416,485.09,4.638303e+07,0.00,01-09-2023,1693515600
1,NVDA,464.595,502.6600,403.110,493.55,1.363638e+09,0.00,31-08-2023,1693429200
2,NVDA,425.170,480.8800,413.460,467.29,8.709012e+08,0.00,31-07-2023,1690750800
3,NVDA,384.890,439.9000,373.560,423.02,1.052802e+09,0.04,30-06-2023,1688072400
4,NVDA,278.400,419.3800,272.400,378.34,1.169407e+09,0.00,31-05-2023,1685480400
5,NVDA,275.090,281.1000,262.200,277.49,7.426625e+08,0.00,28-04-2023,1682629200
6,NVDA,231.920,278.3431,222.970,277.77,1.126658e+09,0.04,31-03-2023,1680210000
7,NVDA,196.910,238.8800,196.110,232.16,1.039409e+09,0.00,28-02-2023,1677535200
8,NVDA,148.510,206.2800,140.340,195.37,9.454958e+08,0.00,31-01-2023,1675116000
9,NVDA,169.990,187.9000,138.840,146.14,8.946151e+08,0.00,30-12-2022,1672351200


In [24]:
temp_stock_df.to_sql('feature_store', conn, index=False, if_exists='replace')

19